# Inception_V3 model summary

In [1]:
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF

gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)  
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)) 

c:\users\czho9\anaconda3\envs\tsf\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
from keras.applications.inception_v3 import InceptionV3
model = InceptionV3()

# Build fine-tune Inception_V3 model

## Preprocessing: load image

In [5]:
import numpy as np
import time
from keras.applications.inception_v3 import InceptionV3 #CHANGE
from keras.preprocessing import image
from keras.applications.inception_v3 import preprocess_input
from keras.applications.inception_v3 import decode_predictions
from keras.layers import Dense,Activation,Flatten
from keras.layers import merge,Input
from keras.models import Model
from keras.utils import np_utils
from sklearn.utils import resample #CHANGE
from sklearn.cross_validation import train_test_split
import os
from sklearn.utils import shuffle

data_path=r"D:\capstone\FGVC_similar_10class"
data = os.listdir(data_path)

In [9]:
for dataset in data:
    img_list = os.listdir(data_path+'/'+dataset)
    print('loaded images'+'{}\n'.format(dataset))
    for img in img_list:
        img_path = data_path+'/'+dataset+'/'+img
        #load image
        img = image.load_img(img_path,target_size=(299,299)) 
        #image preprocess
        x = image.img_to_array(img)
        x = np.expand_dims(x,axis=0)
        x = preprocess_input(x)
        img_data_list.append(x)
                
img_data = np.array(img_data_list)
print(img_data.shape)
img_data = np.rollaxis(img_data,1,0)
print(img_data.shape)
img_data = img_data[0]
print(img_data.shape)

num_classes = 10
num_of_samples = img_data.shape[0]
labels = np.ones((num_of_samples,),dtype='int64')

loaded imagesSetophaga americana

loaded imagesSetophaga coronata

loaded imagesSetophaga coronata auduboni

loaded imagesSetophaga coronata coronata

loaded imagesSetophaga magnolia

loaded imagesSetophaga palmarum

loaded imagesSetophaga petechia

loaded imagesSetophaga ruticilla

loaded imagesSetophaga townsendi

loaded imagesSetophaga virens

(6952, 1, 299, 299, 3)
(1, 6952, 299, 299, 3)
(6952, 299, 299, 3)


In [10]:
labels[0:200] = 0
labels[223:423] =1
labels[1899:2099] =2
labels[2245:2445]=3
labels[2461:2661]=4
labels[2707:2907]=5
labels[3008:3208]=6
labels[3708:3908]=7
labels[4064:4264]=8
labels[4428:4628]=9

names=['Setophaga americana', 'Setophaga coronata', 'Setophaga coronata auduboni', 'Setophaga coronata coronata', 'Setophaga magnolia', 'Setophaga palmarum', 'Setophaga petechia', 'Setophaga ruticilla', 'Setophaga townsendi', 'Setophaga virens']


## Preprocessing: split into train and test set 

In [11]:
Y = np_utils.to_categorical(labels,num_classes)

x,y = shuffle(img_data,Y,random_state=2) 

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2)

## Fine-tune model summary

In [12]:
image_input = Input(shape=(299,299,3))
model=InceptionV3(input_tensor=image_input,weights='imagenet')
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d_189 (Conv2D)             (None, 149, 149, 32) 864         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_189 (BatchN (None, 149, 149, 32) 96          conv2d_189[0][0]                 
__________________________________________________________________________________________________
activation_189 (Activation)     (None, 149, 149, 32) 0           batch_normalization_189[0][0]    
__________________________________________________________________________________________________
conv2d_190

In [13]:
last_layer = model.get_layer('avg_pool').output
out =Dense(num_classes,activation='softmax',name='output')(last_layer)
custom_InceptionV3_model = Model(image_input,out)
custom_InceptionV3_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d_189 (Conv2D)             (None, 149, 149, 32) 864         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_189 (BatchN (None, 149, 149, 32) 96          conv2d_189[0][0]                 
__________________________________________________________________________________________________
activation_189 (Activation)     (None, 149, 149, 32) 0           batch_normalization_189[0][0]    
__________________________________________________________________________________________________
conv2d_190

# Train the fine-tune Inception_V3 model

In [14]:
for layer in custom_InceptionV3_model.layers[:-1]:
    layer.trainable = False
    
print(custom_InceptionV3_model.layers[3].trainable)

custom_InceptionV3_model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

t=time.time()
hist=custom_InceptionV3_model.fit(X_train,y_train,batch_size=20,epochs=8,verbose=1,validation_data=(X_test,y_test))
print('Taining time:%s' %(t-time.time()))
(loss,accuracy)=custom_InceptionV3_model.evaluate(X_test,y_test,batch_size=10,verbose=1)
print('loss={:.4f},accuracy:{:.4f}%'.format(loss,accuracy*100))

False
Train on 5561 samples, validate on 1391 samples
Epoch 1/8
5561/5561 [==============================] - 170s 31ms/step - loss: 1.2296 - acc: 0.7333 - val_loss: 1.2216 - val_acc: 0.7405
Epoch 2/8
5561/5561 [==============================] - 149s 27ms/step - loss: 1.1435 - acc: 0.7348 - val_loss: 1.2316 - val_acc: 0.7584
Epoch 3/8
5561/5561 [==============================] - 147s 26ms/step - loss: 1.0981 - acc: 0.7353 - val_loss: 1.1876 - val_acc: 0.7584
Epoch 4/8
5561/5561 [==============================] - 145s 26ms/step - loss: 1.0681 - acc: 0.7348 - val_loss: 1.6586 - val_acc: 0.3336
Epoch 5/8
5561/5561 [==============================] - 148s 27ms/step - loss: 1.0550 - acc: 0.7304 - val_loss: 1.5204 - val_acc: 0.4831
Epoch 6/8
5561/5561 [==============================] - 150s 27ms/step - loss: 1.0234 - acc: 0.7349 - val_loss: 1.6832 - val_acc: 0.3961
Epoch 7/8
5561/5561 [==============================] - 148s 27ms/step - loss: 1.0170 - acc: 0.7349 - val_loss: 1.1050 - val_acc: 0

In [9]:
custom_InceptionV3_model.save('Inception_V3_10epoch_1.h5')

# Predict the fine-tune Inception_V3 model

In [15]:
from sklearn.metrics import classification_report,confusion_matrix

In [16]:
Y_pred=custom_InceptionV3_model.predict(X_test)
y_pred=np.argmax(Y_pred,axis=1)

## Activation Map

In [23]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
from keras.models import load_model
from keras.preprocessing import image
from keras import backend as K

In [36]:
data_path=r"D:\capstone\FGVC_10_similar_classes)"
class_labels = os.listdir(data_path)
print(class_labels)
#model = load_model('Inception_V3_10epoch_1.h5')
class_to_label = {c:i for i,c in enumerate(class_labels)}

['Calidris alba', 'Gallus gallus domesticus', 'Geococcyx californianus', 'Phoenicopterus roseus', 'Picoides villosus', 'Spheniscus demersus', 'Sterna striata', 'Struthio camelus', 'Thryothorus ludovicianus', 'Tyrannus verticalis']


In [37]:
# Last convolutional layer name, search it first using model.summary()
last_conv_layer_name = r'mixed10'
# Dimension of images used to train the model (after resizing)
train_model_width = 299
train_model_height = 299

In [50]:
# The main function
# Taken from the Deep Learning with Python textbook, with a little bit of modification to generalize it
def generate_prediction_heatmap(model, class_labels, 
                                train_model_width, train_model_height, 
                                last_conv_layer_name,
                                input_image_path,
                                output_image_dir,
                                heatmap_class_label=-1,
                                verbose=True):
    if verbose:
        print('Loading image from:', input_image_path)
    img = image.load_img(input_image_path, target_size=(train_model_height,train_model_width))

    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = x/255
    
    y_prob = model.predict(x)
    y_class = y_prob.argmax(axis=-1)[0]
    if verbose:
        print('-'*20)
        for i,prob in enumerate(y_prob[0]):
            print('Class:',class_labels[i],', probability:','{:.4f}'.format(prob))
            print(i)
        print('-'*20)
        print('Predicted :', class_labels[y_class], 'as the best class')
    # Use the supplied label to generate the heatmap if supplied, use the best class otherwise
    y_class = y_class if heatmap_class_label == -1 else heatmap_class_label

    out = model.output[:,y_class]
    last_conv_layer = model.get_layer(last_conv_layer_name)
    print(last_conv_layer)
    grads = K.gradients(out, last_conv_layer.output)[0]
    print(grads)
    pooled_grads = K.mean(grads, axis=(0,1,2))
    iterate = K.Function([model.input], [pooled_grads, last_conv_layer.output[0]])
    pooled_grads_value, conv_layer_output_value = iterate([x])
    for i in range(last_conv_layer.output_shape[-1]):
        conv_layer_output_value[:,:,i] *= pooled_grads_value[i]

    heatmap = np.mean(conv_layer_output_value, axis=-1)
    heatmap = np.maximum(heatmap,0)
    heatmap /= np.max(heatmap)  

    img = cv2.imread(input_image_path)
    # Need to be converted due to weird colour ordering of the image
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    superimposed_img = heatmap * 0.4 + img
    
    output_image_path = os.path.join(output_image_dir, os.path.basename(input_image_path))
    if verbose:
        print('Generating output heatmap image at:',output_image_path)
        print()
    cv2.imwrite(output_image_path, superimposed_img)